# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 10 Assignment: Time Series Neural Network**

**Student Name: Rebekah Griesenauer**

# Assignment Instructions

For this assignment you will use a LSTM to predict a time series contained in the data file **[series-31-spring-2018.csv](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/data/series-31-spring-2018.csv)**.  The code that you will use to complete this will be similar to the sunspots example from the course module.  This data set contains two columns: *time* and *value*.  Create a LSTM network and train it with a sequence size of 5 and a prediction window of 1.  If you use a different sequence size, you will not have the correct number of submission rows. Train the neural network, the data set is fairly simple and you should easily be able to get a RMSE below 1.0.  FYI, I generate this datasets by fitting a cubic spline to a series of random points. 

This is a time series data set, do not randomize the order of the rows!  For your training data use all *time* values less than 3000 and for test, use the remaining values greater than or equal to 3000. For the submit file, send me the results of your test evaluation.  You should have two columns: *time* and *value*.  The column *time* should be the time at the beginning of each predicted sequence. The *value* should be the next value that was predicted for each of your sequences.

Your submission file will look similar to:

# Helpful Functions

You will see these at the top of every module and assignment.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [1]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #10 Sample Code

The following code provides a starting point for this assignment.

In [2]:
import numpy as np
def to_sequences(seq_size, obs):
    x = []
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE-1):
        #print(i)
        window = obs[i:(i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)
        
    return np.array(x),np.array(y)
    

key = "r7Th7Lhul2aj0C93lQNZkA9QZFuc6RW1YfPO4QM2"  # This is an example key and will not work.
file = '/Users/rebekahgriesenauer/Documents/AI/deep_neural_networks/assignments/assignment_griesenauer_class10.ipynb'

# Read from time series file
path = "../data/"
filename = os.path.join(path,"series-31-spring-2018.csv")    
df = pd.read_csv(filename) 

print("Starting file:")
print(df[0:10])

print("Ending file:")
print(df[-10:])

Starting file:
   time      value
0     0  10.000000
1     1  10.050953
2     2  10.101758
3     3  10.152415
4     4  10.202924
5     5  10.253286
6     6  10.303499
7     7  10.353566
8     8  10.403485
9     9  10.453256
Ending file:
      time      value
3990  3990  14.694572
3991  3991  14.727313
3992  3992  14.760351
3993  3993  14.793687
3994  3994  14.827322
3995  3995  14.861256
3996  3996  14.895491
3997  3997  14.930026
3998  3998  14.964862
3999  3999  15.000000


In [36]:
SEQUENCE_SIZE = 5
df_train = df[df['time']<3000]
df_test = df[df['time']>=3000]

value_train = df_train['value'].tolist()
value_test = df_test['value'].tolist()

print("Training set has {} observations.".format(len(value_train)))
print("Test set has {} observations.".format(len(value_test)))


x_train,y_train = to_sequences(SEQUENCE_SIZE,value_train)
x_test,y_test = to_sequences(SEQUENCE_SIZE,value_test)

print("Shape of training set: {}".format(x_train.shape))
print("Shape of test set: {}".format(x_test.shape))

Training set has 3000 observations.
Test set has 1000 observations.
Shape of training set: (2994, 5, 1)
Shape of test set: (994, 5, 1)


In [37]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
import numpy as np

print('Build model...')
model = Sequential()
model.add(LSTM(64, dropout=0.0, recurrent_dropout=0.0,input_shape=(None, 1)))
model.add(Dense(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
print('Train...')

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Build model...
Train...
Train on 2994 samples, validate on 994 samples
Epoch 1/1000
 - 1s - loss: 66.1262 - val_loss: 8.8581
Epoch 2/1000
 - 1s - loss: 0.2310 - val_loss: 0.3108
Epoch 3/1000
 - 1s - loss: 0.0149 - val_loss: 0.1188
Epoch 4/1000
 - 1s - loss: 0.0087 - val_loss: 0.0736
Epoch 5/1000
 - 1s - loss: 0.0049 - val_loss: 0.0487
Epoch 6/1000
 - 1s - loss: 0.0030 - val_loss: 0.0372
Epoch 7/1000
 - 1s - loss: 0.0025 - val_loss: 0.0427
Epoch 8/1000
 - 1s - loss: 0.0031 - val_loss: 0.0268
Epoch 9/1000
 - 1s - loss: 0.0038 - val_loss: 0.0244
Epoch 10/1000
 - 1s - loss: 0.0048 - val_loss: 0.0270
Epoch 11/1000
 - 1s - loss: 0.0016 - val_loss: 0.0239
Epoch 12/1000
 - 1s - loss: 0.0024 - val_loss: 0.0236
Epoch 13/1000
 - 1s - loss: 0.0016 - val_loss: 0.0160
Epoch 14/1000
 - 1s - loss: 0.0018 - val_loss: 0.0248
Epoch 15/1000
 - 1s - loss: 0.0020 - val_loss: 0.0323
Epoch 16/1000
 - 1s - loss: 0.0015 - val_loss: 0.0279
Epoch 17/1000
 - 1s - loss: 0.0015 - val_loss: 0.0236
Epoch 18/1000
 - 1s

In [38]:
from sklearn import metrics

pred = model.predict(x_test) 
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

Score (RMSE): 0.15378220314957855


In [39]:
df_test_trim = df_test[SEQUENCE_SIZE+1:]
len(df_test_trim['time'])
dfSubmit=pd.DataFrame()
dfSubmit['time']=df_test_trim['time']
dfSubmit['value']=pred


In [42]:
dfSubmit.set_index('time')

,value
time,
3006,36.496433
3007,36.503685
3008,36.510773
3009,36.517708
3010,36.524483
3011,36.531090
3012,36.537476
3013,36.543682
3014,36.549591


In [43]:
# Submit this assignment
submit(source_file=file,data=dfSubmit,key=key,no=10)

Success: Submitted Assignment #10 for rebekah.griesenauer:
You have submitted this assignment 5 times. (this is fine)

